In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
!pip install tensorflow==1.12

     |████████████████████████████████| 83.1MB 349kB/s 
     |████████████████████████████████| 3.1MB 29.9MB/s 
  Found existing installation: tensorboard 1.14.0
    Uninstalling tensorboard-1.14.0:
      Successfully uninstalled tensorboard-1.14.0
  Found existing installation: tensorflow 1.14.0
    Uninstalling tensorflow-1.14.0:
      Successfully uninstalled tensorflow-1.14.0


In [0]:
import os
from io import BytesIO
from PIL import Image

import cv2
import tensorflow as tf

def del_all_flags(FLAGS):
    flags_dict = FLAGS._flags()    
    keys_list = [keys for keys in flags_dict]    
    for keys in keys_list:
        FLAGS.__delattr__(keys)

del_all_flags(tf.flags.FLAGS)


FLAGS = tf.app.flags
FLAGS.DEFINE_string('data_dir',
                    '/content/gdrive/My Drive/datasetSA',
                    'Path to the input data')
FLAGS.DEFINE_string('train_output_path',
                    'train.tfrecord',
                    'Path to output train TFRecord')
FLAGS.DEFINE_string('eval_output_path',
                    'eval.tfrecord',
                    'Path to output eval TFRecord')
FLAGS.DEFINE_float(
    'train_eval_split_factor', 0.75,
    'use this factor to split the train (default 3/4) and '
    'eval data (default 1/4) in data_dir')
FLAGS.DEFINE_integer('width', 120, 'customize image width')
FLAGS.DEFINE_integer('height', 100, 'customize image height')
FLAGS.DEFINE_integer('channel', 3, 'image color channel')
FLAGS.DEFINE_integer('skip_frames', 10,
                     'the number of frames we skip when we process the video')
FLAGS.DEFINE_integer('num_frames_per_clip', 16,
                     'the number of frames for a video clip')
FLAGS = FLAGS.FLAGS



def get_clips(image_list):
    # Given a list of images, return video clips of (num_frames_per_clip) consecutive frames as a list.
    video_clips = []
    images_len = len(image_list)
    if images_len < FLAGS.num_frames_per_clip:
        return video_clips

    # Prepare the first clip
    video_clips.append(image_list[:FLAGS.num_frames_per_clip])

    num_of_extra_clip = int(
        (images_len - FLAGS.num_frames_per_clip) / FLAGS.skip_frames)
    for i in range(1, num_of_extra_clip + 1):
        start = i * FLAGS.skip_frames - 1
        end = start + FLAGS.num_frames_per_clip
        video_clips.append(image_list[start:end])

    return video_clips

def process_dataset(train_writer, eval_writer, data_dir):
    label = 0
    # [class1, class2, class3, ..., class n]
    for class_dir in os.listdir(data_dir):
        class_path = os.path.join(data_dir, class_dir)
        if os.path.isdir(class_path):
            # Set the label value for this class, start from 0
            label += 1
            print("Processing class: " + str(label))
            # Process each video file in this class
            video_filenames = os.listdir(class_path)
            for video_filename in video_filenames[0:int(
                    FLAGS.train_eval_split_factor * len(video_filenames))]:
                process_video(train_writer, class_path, video_filename, label)
            for video_filename in video_filenames[
                    int(FLAGS.train_eval_split_factor *
                        len(video_filenames)):len(video_filenames)]:
                process_video(eval_writer, class_path, video_filename, label)


def process_video(writer, class_path, video_filename, label):
    video_filename_path = os.path.join(class_path, video_filename)
    if video_filename_path.endswith('mp4'):
        video_clips = _convert_video_to_clips(video_filename_path)
        # Convert the clip to tf record
        for clip in video_clips:
            tf_example = create_tf_example(raw=clip, label=label)
            writer.write(tf_example.SerializeToString())


def _convert_video_to_clips(video_path):
    # Use opencv to read video to list of images
    video_images_list = []
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        # frame shape [height, width, channel]
        _, frame = cap.read()
        try:
            # pil_image shape [width, height, channel]
            pil_image = Image.fromarray(frame)
            # Resize the image and convert the image according to the channel information
            if FLAGS.channel == 1:
                pil_image = pil_image.resize((FLAGS.width, FLAGS.height),
                                             Image.NEAREST).convert('L')
            else:
                pil_image = pil_image.resize((FLAGS.width, FLAGS.height),
                                             Image.NEAREST)
            # Encode the image to JPEG
            with BytesIO() as buffer:
                pil_image.save(buffer, format="JPEG")
                video_images_list.append(buffer.getvalue())
        except AttributeError:
            # Fail to read the image
            break

    # Convert list of images to clips of images with type np.float32
    return get_clips(image_list=video_images_list)


def _bytelist_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))


def _bytes_feature(value):
    return tf.train.Feature(bytes_list=tf.train.BytesList(value=value))


def _int64_feature(value):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))


def create_tf_example(raw, label):
    return tf.train.Example(
        features=tf.train.Features(
            feature={
                'clip/width': _int64_feature(FLAGS.width),
                'clip/height': _int64_feature(FLAGS.height),
                'clip/channel': _int64_feature(FLAGS.channel),
                'clip/raw': _bytelist_feature(raw),
                'clip/label': _int64_feature(label)
            }))


def get_total_video_clip_number(data_path):
    count = 0
    for _ in tf.python_io.tf_record_iterator(data_path):
        count += 1
    return count


def main(_):
    # Write the dataset
    train_writer = tf.python_io.TFRecordWriter(FLAGS.train_output_path)
    eval_writer = tf.python_io.TFRecordWriter(FLAGS.eval_output_path)

    process_dataset(
        train_writer=train_writer,
        eval_writer=eval_writer,
        data_dir=FLAGS.data_dir)

    train_writer.close()
    eval_writer.close()

    # Count the dataset record
    print("Total clips in train dataset: " +
          str(get_total_video_clip_number(FLAGS.train_output_path)))
    print("Total clips in eval dataset: " +
          str(get_total_video_clip_number(FLAGS.eval_output_path)))


#if __name__ == '__main__':
#    tf.app.run()


In [7]:
class DenseNet3D(object):
    """3D DenseNet Model class"""

    def __init__(
            self,
            video_clips,  # Shape: [batch_size, sequence_length, height, width, channels]
            labels,  # Shape: [batch_size, num_classes] 
            initial_learning_rate,
            decay_step,
            lr_decay_factor,
            num_classes,
            growth_rate,
            network_depth,
            total_blocks,
            keep_prob,
            weight_decay,
            reduction,
            bc_mode=False,
            **kwargs):
        self.video_clips = video_clips
        self.labels = labels
        self.num_classes = num_classes
        self.growth_rate = growth_rate
        self.network_depth = network_depth

        # How many features will be received after first convolution value
        self.first_output_features = growth_rate * 2

        self.total_blocks = total_blocks
        self.layers_per_block = (network_depth -
                                 (total_blocks + 1)) // total_blocks

        # Compression rate at the transition layers
        self.reduction = reduction
        self.bc_mode = bc_mode
        if not bc_mode:
            self.reduction = 1.0
            print(
                "Build 3D DenseNet model with %d blocks, %d composite layers each."
                % (total_blocks, self.layers_per_block))

        if bc_mode:
            self.layers_per_block = self.layers_per_block // 2
            print(
                "Build 3D DenseNet-BC model with %d blocks, %d bottleneck layers and %d composite layers each."
                % (total_blocks, self.layers_per_block, self.layers_per_block))

        self.keep_prob = keep_prob
        self.weight_decay = weight_decay
        self._is_training = tf.convert_to_tensor(True)

        # Initialize the global step
        self.global_step = tf.train.get_or_create_global_step()

        self.learning_rate = tf.train.exponential_decay(
            initial_learning_rate,
            self.global_step,
            decay_step,
            lr_decay_factor,
            staircase=True)

        self._build_graph()

    def _build_graph(self):
        # First convolution layer
        with tf.variable_scope('Initial_convolution'):
            output = self._conv3d(
                self.video_clips,
                out_features_count=self.first_output_features,
                kernel_size=7,
                strides=[1, 1, 2, 2, 1])
            output = self._pool(output, k=3, d=2, k_stride=2, d_stride=1)

        # Add 3D DenseNet blocks
        for block in range(self.total_blocks):
            with tf.variable_scope("Block_%d" % block):
                output = self._add_block(output, self.growth_rate,
                                         self.layers_per_block)
            # The last block exist without transition layer
            if block != self.total_blocks - 1:
                with tf.variable_scope("Transition_after_block_%d" % block):
                    output = self._transition_layer(output, pool_depth=2)

        # Fully connected layers
        with tf.variable_scope('Transition_to_classes'):
            self._logits = self._trainsition_layer_to_classes(output)

        # Prediction result
        self._prediction = tf.argmax(self._logits, 1)

        # Losses
        self._cross_entropy = tf.reduce_mean(
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                logits=self._logits, labels=self.labels),
            name='Cross_entropy')
        self.l2_loss = tf.add_n(
            [tf.nn.l2_loss(var) for var in tf.trainable_variables()])
        self.total_loss = self._cross_entropy + self.l2_loss * self.weight_decay

        # Optimizer and training op
        self._train_op = tf.contrib.layers.optimize_loss(
            loss=self.total_loss,
            global_step=self.global_step,
            learning_rate=self.learning_rate,
            optimizer='Momentum')

    @property
    def logits(self):
        return self._logits

    @property
    def train_op(self):
        return self._train_op

    @property
    def losses(self):
        return self._cross_entropy

    @property
    def prediction(self):
        return self._prediction

    @property
    def accuracy(self):
        correct_prediction = tf.equal(tf.argmax(self._logits, 1), self.labels)
        return tf.metrics.mean(tf.cast(correct_prediction, tf.float32))

    @property
    def is_training(self):
        return self._is_training

    @is_training.setter
    def is_training(self, value):
        self._is_training = tf.convert_to_tensor(value)

    def _conv3d(self,
                inputs,
                out_features_count,
                kernel_size,
                strides=[1, 1, 1, 1, 1],
                padding='SAME'):
        input_features_count = int(inputs.get_shape()[-1])
        kernel = tf.get_variable(
            'kernel',
            shape=[
                kernel_size, kernel_size, kernel_size, input_features_count,
                out_features_count
            ],
            initializer=tf.random_normal_initializer())
        with tf.name_scope('3d_conv'):
            return tf.nn.conv3d(
                inputs, filter=kernel, strides=strides, padding=padding)

    def _pool(self,
              inputs,
              k,
              d=2,
              k_stride=None,
              d_stride=None,
              width_k=None,
              k_stride_width=None):
        if not width_k:
            width_k = k
        kernel_size = [1, d, k, width_k, 1]
        if not k_stride:
            k_stride = k
        if not k_stride_width:
            k_stride_width = k_stride
        if not d_stride:
            d_stride = d
        strides = [1, d_stride, k_stride, k_stride_width, 1]
        return tf.nn.max_pool3d(
            inputs, ksize=kernel_size, strides=strides, padding='SAME')

    def _add_block(self, inputs, growth_rate, layers_per_block):
        for layer in range(layers_per_block):
            with tf.variable_scope("layer_%d" % layer):
                return self._add_internal_layer(inputs, growth_rate)

    def _add_internal_layer(self, inputs, growth_rate):
        if not self.bc_mode:
            composite_out = self._composite_function(
                inputs, out_features_count=growth_rate, kernel_size=3)
        elif self.bc_mode:
            bottleneck_out = self._bottleneck(
                inputs, out_features_count=growth_rate)
            composite_out = self._composite_function(
                bottleneck_out, out_features_count=growth_rate, kernel_size=3)

        with tf.name_scope('concat'):
            return tf.concat(axis=4, values=(inputs, composite_out))

    def _composite_function(self, inputs, out_features_count, kernel_size):
        with tf.variable_scope('composite_function'):
            # Batch normalization
            output = self._batch_norm(inputs)
            # ReLU
            with tf.name_scope('ReLU'):
                output = tf.nn.relu(output)
            # Convolution
            output = self._conv3d(
                output,
                out_features_count=out_features_count,
                kernel_size=kernel_size)
            # Dropout
            output = self._dropout(output)
        return output

    def _bottleneck(self, inputs, out_features_count):
        with tf.variable_scope('bottleneck'):
            # Batch normalization
            output = self._batch_norm(inputs)
            # ReLU
            with tf.name_scope('ReLU'):
                output = tf.nn.relu(output)

            inter_features = out_features_count * 4
            output = self._conv3d(
                output,
                out_features_count=inter_features,
                kernel_size=1,
                padding='VALID')
            output = self._dropout(output)
        return output

    def _batch_norm(self, inputs):
        with tf.name_scope('batch_normalization'):
            output = tf.contrib.layers.batch_norm(
                inputs,
                scale=True,
                is_training=self._is_training,
                updates_collections=None)
        return output

    def _dropout(self, inputs):
        if self.keep_prob < 1:
            with tf.name_scope('dropout'):
                output = tf.cond(self._is_training,
                                 lambda: tf.nn.dropout(inputs, self.keep_prob),
                                 lambda: inputs)
        else:
            output = inputs
        return output

    def _transition_layer(self, inputs, pool_depth):
        out_features_count = int(int(inputs.get_shape()[-1]) * self.reduction)
        output = self._composite_function(
            inputs, out_features_count=out_features_count, kernel_size=1)
        with tf.name_scope('pooling'):
            output = self._pool(output, k=2, d=pool_depth)
        return output

    def _trainsition_layer_to_classes(self, inputs):
        # Batch normalization
        output = self._batch_norm(inputs)
        # ReLU
        with tf.name_scope('ReLU'):
            output = tf.nn.relu(output)
        # pooling
        last_pool_kernel_width = int(output.get_shape()[-2])
        last_pool_kernel_height = int(output.get_shape()[-3])
        last_sequence_length = int(output.get_shape()[1])
        with tf.name_scope('pooling'):
            output = self._pool(
                output,
                k=last_pool_kernel_height,
                d=last_sequence_length,
                width_k=last_pool_kernel_width,
                k_stride_width=last_pool_kernel_width)
        # Fully connected
        features_total = int(output.get_shape()[-1])
        output = tf.reshape(output, [-1, features_total])
        weight = tf.get_variable(
            'fc_w',
            shape=[features_total, self.num_classes],
            initializer=tf.contrib.layers.xavier_initializer())
        bias = tf.get_variable(
            'fc_bias',
            shape=[self.num_classes],
            initializer=tf.zeros_initializer())
        logits = tf.matmul(output, weight) + bias
        return logits

def model_fn(features, labels, mode, params):
    # Define the model
    model = DenseNet3D(
        video_clips=features['video_clips'], labels=labels, **params)

    # Get the prediction result
    if mode == tf.estimator.ModeKeys.PREDICT:
        model.is_training = False
        return _predict_result(model.logits)

    return tf.estimator.EstimatorSpec(
        mode=mode,
        loss=model.losses,
        train_op=model.train_op,
        eval_metric_ops={'eval_accuracy': model.accuracy})


def _predict_result(model):
    predictions = {'probabilities': model.prediction, 'logits': model.logits}
    return tf.estimator.EstimatorSpec(
        mode=tf.estimator.ModeKeys.PREDICT, predictions=predictions)


def serving_input_fn(params):
    inputs = {
        'video_clips':
        tf.placeholder(
            tf.float32,
            shape=[
                None, params['num_frames_per_clip'], params['height'],
                params['width'], params['channel']
            ])
    }
    return tf.estimator.export.build_raw_serving_input_receiver_fn(inputs)()


def train_input_fn(training_dir, params):
    directory = os.path.join(training_dir, 'train.tfrecord')
    return _build_tfrecord_dataset(directory, params['train_total_video_clip'],
                                   **params)


def eval_input_fn(evaluating_dir, params):
    directory = os.path.join(evaluating_dir, 'eval.tfrecord')
    return _build_tfrecord_dataset(directory, params['eval_total_video_clip'],
                                   **params)


def _build_tfrecord_dataset(directory, total_clip_num, batch_size, **params):
    '''
    Buffer the training dataset to TFRecordDataset with the following video shape
    [num_frames_per_clip, height, width, channel]
    ex: [16, 100, 120, 3]
    '''
    dataset = tf.data.TFRecordDataset(directory)
    dataset = dataset.shuffle(buffer_size=total_clip_num)
    dataset = dataset.map(
        map_func=
        lambda serialized_example: _parser(serialized_example, **params))
    dataset = dataset.repeat()
    iterator = dataset.batch(batch_size=batch_size).make_one_shot_iterator()
    clips, labels = iterator.get_next()
    return {'video_clips': clips}, labels


def _parser(serialized_example, num_frames_per_clip, **params):
    features = tf.parse_single_example(
        serialized_example,
        features={
            'clip/width': tf.FixedLenFeature([], tf.int64),
            'clip/height': tf.FixedLenFeature([], tf.int64),
            'clip/channel': tf.FixedLenFeature([], tf.int64),
            'clip/raw': tf.FixedLenFeature([num_frames_per_clip], tf.string),
            'clip/label': tf.FixedLenFeature([], tf.int64)
        })

    def mapping_func(image):
        return _decode_image(image, **params)

    clip = tf.map_fn(mapping_func, features['clip/raw'], dtype=tf.float32)
    return clip, features['clip/label']


def _decode_image(image, channel, width, height, **params):
    image = tf.image.decode_jpeg(image, channels=channel)
    # This set_shape step is necesary for the last trainsition_layer_to_classes layer in the model
    image.set_shape([height, width, channel])
    image = tf.cast(image, tf.float32)
    return image

"""This is the main class"""


MODEL_DIR = 'denseNet3d_result'
DATA_DIR = '/content'

HYPERPARAMETERS = {
    'num_classes': 6,  # The number of the classes that this dataset had
    'batch_size': 20,
    'initial_learning_rate': 0.1,
    'decay_step': 5000,
    'lr_decay_factor':
    0.1,  # Learning rate will decay by a factor for every decay_step
    'growth_rate': 12,  # Grows rate for every layer [12, 24, 40]
    'network_depth': 20,  # Depth of the whole network [20, 40, 250]
    'total_blocks': 3,  # Total blocks of layers stack
    'keep_prob': 0.9,  # Keep probability for dropout
    'weight_decay': 1e-4,  # Weight decay for L2 loss
    'model_type': 'DenseNet3D',
    'reduction': 0.5,  # Reduction rate at transition layer for the models
    'bc_mode': True,
    'num_frames_per_clip': 16,  # The length of the video clip
    'width': 120,
    'height': 100,
    'channel': 3,
    'train_total_video_clip': 4183,
    'eval_total_video_clip': 914
}

TFRUNCONFIG = tf.estimator.RunConfig(
    log_step_count_steps=1, save_summary_steps=1, model_dir=MODEL_DIR)

CLASSIFIER = tf.estimator.Estimator(
    model_fn=model_fn, params=HYPERPARAMETERS, config=TFRUNCONFIG)

CLASSIFIER.train(
    input_fn=lambda: train_input_fn(DATA_DIR, HYPERPARAMETERS),
    steps=210)

CLASSIFIER.evaluate(
    input_fn=lambda: eval_input_fn(DATA_DIR, HYPERPARAMETERS),
    steps=45)



INFO:tensorflow:Using config: {'_model_dir': 'denseNet3d_result', '_tf_random_seed': None, '_save_summary_steps': 1, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 1, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7eff020b3160>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
Build 3D DenseNet-BC model with 3 blocks, 2 bottleneck layers and 2 composite layers each.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Cr

{'eval_accuracy': 0.35444444, 'global_step': 4620, 'loss': 1.6496755}